In [ ]:
# Funciones de detección de adicionales. Cada una genera 1 columna
def tiene_SUM(df):
  df['tiene_SUM'] = df.description.str.contains('sum', regex = True)
  return df
def tiene_pileta(df):
  df['tiene_pileta'] = df.description.str.contains('pileta', regex = True)
  return df
def tiene_laundry(df):
  df['tiene_laundry'] = df.description.str.contains('laundry', regex = True)
  return df
def tiene_gimnasio(df):
  df['tiene_gimnasio'] = df.description.str.contains('gimnasio', regex = True)
  return df
def tiene_cochera(df):
  df['tiene_cochera'] = df.description.str.contains('cochera', regex = True)
  return df

# columnas generadas: 'tiene_SUM', 'tiene_pileta', 'tiene_laundry', 'tiene_gimnasio', 'tiene_cochera'

In [ ]:
# Detección de posible pozo. Puede ser perfeccionada
def posible_pozo(df):
  df['posible_pozo'] = df.description.str.contains('pozo', regex = True)
  return df
# columnas generadas: 'posible_pozo'

In [ ]:
# Detección de antiguedad en texto libre
def antiguedad_detectada(df):
  df['antiguedad_detectada'] = df.description.str.contains('pozo', regex = True)
  return df